In [1]:
#Standard Header used on the projects

#first the major packages used for math and graphing
import numpy as np
import matplotlib.pyplot as plt
from cycler import cycler
import scipy.special as sp
#import pandas as pd

#Custome graph format style sheet
#plt.style.use('Prospectus.mplstyle')

#If being run by a seperate file, use the seperate file's graph format and saving paramaeters
#otherwise set what is needed
if not 'Saving' in locals():
    Saving = True
if not 'Titles' in locals():
    Titles = True
if not 'Ledgends' in locals():
    Ledgends = True
if not 'FFormat' in locals():
    FFormat = '.png'
Computer = "SciClone"   # "SciClone"    "WinLap"  "LinLap"   "Desktop"  "PortLap"

#Standard cycle for collors and line styles
default_cycler = (cycler('color', ['0.00', '0.40', '0.60', '0.70']) + cycler(linestyle=['-', '--', ':', '-.']))
plt.rc('axes', prop_cycle=default_cycler)
my_cmap = plt.get_cmap('gray')

import datetime
import multiprocessing
from joblib import Parallel, delayed
import pywt
import os as os
import statistics as st
import os as os
import random
import time
import platform
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import cv2
from sklearn.model_selection import train_test_split

tf.config.set_visible_devices([], 'GPU')

from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import  EarlyStopping
from tensorflow.keras.models import Sequential, Model 


In [2]:
HostName = platform.node()
dataSize = 'big'

if HostName == "Server":
    Computer = "Desktop"   
elif HostName[-6:] == 'wm.edu':
    Computer = "SciClone"
elif HostName == "SchoolLaptop":
    Computer = "LinLap"
elif HostName == "WTC-TAB-512":
    Computer = "PortLap"
else:
    Computer = "WinLap"

if Computer == "SciClone":
    location = '/sciclone/home20/dchendrickson01/'
elif Computer == "WinLap":
    location = 'C:\\Data\\'
elif Computer == "Desktop":
    location = "E:\\Backups\\Dan\\CraneData\\"
elif Computer == "LinLap":
    location = '/home/dan/Output/'

if Computer ==  "SciClone":
    rootfolder = '/sciclone/home20/dchendrickson01/'
    if dataSize == 'big':
        folder = '/sciclone/scr10/dchendrickson01/CraneData/'
        imFolder ='/sciclone/scr10/dchendrickson01/BigPrints/'
    else:
        folder = '/sciclone/data10/dchendrickson01/SmallCopy/'
elif Computer == "Desktop":
    rootfolder = location
    if dataSize == 'big':
        folder = 'G:\\CraneData\\'
    else:
        folder = rootfolder + "SmallCopy\\"
elif Computer =="WinLap":
    rootfolder = location
    folder = rootfolder + "SmallCopy\\"   
elif Computer == "LinLap":
    rootfolder = '/home/dan/Data/'
    folder = rootfolder + 'SmallCopy/'
    
#Pre Calculations
num_cores = multiprocessing.cpu_count()-1


num_classes = 2
input_shape = (500, 10000, 3)

In [3]:
Good_files =[os.path.join(dp, f) for dp, dn, fn in os.walk(os.path.expanduser(imFolder+'Good/')) for f in fn]
Bad_files =[os.path.join(dp, f) for dp, dn, fn in os.walk(os.path.expanduser(imFolder+'Bad/')) for f in fn]

print('Have Files')

Have Files


In [4]:
  
def addImage(file):
    pathname, extension = os.path.splitext(file)
    filetree = pathname.split('/')
    foldername = filetree[-2]
    if foldername == 'Good':
        truth = [0]
        image = cv2.imread(file, flags=cv2.IMREAD_COLOR)
        return truth,image
    elif foldername == 'Bad':
        truth = [1]
        image = cv2.imread(file, flags=cv2.IMREAD_COLOR)
        return truth,image
    else:
        return None


In [5]:
def loopLearning(i, loopSize, model):
    tg = Parallel(n_jobs=num_cores)(delayed(addImage)(file) for file in Good_files[i*loopSize:(i+1)*loopSize])
    tb = Parallel(n_jobs=num_cores)(delayed(addImage)(file) for file in Bad_files[i*loopSize:(i+1)*loopSize])
    
    Images = []
    Truth = []

    for j in tg:
        Truth.append(j[0])
        Images.append(j[1])
    for j in tb:
        Truth.append(j[0])
        Images.append(j[1])
    
    Images=np.asarray(Images)
    Truth=np.asarray(Truth)
    
    x_train, x_test,y_train,y_test = train_test_split(Images,Truth, test_size=0.15, shuffle=True, random_state=0)

    y_train = np.matrix(y_train)
    x_train = np.asarray(x_train)
    
    history = model.fit(x = x_train, y = y_train, epochs=1, batch_size = 32 , shuffle=True, validation_split=0.25, callbacks=callbacks)

    return history, model


In [6]:
img_width = 500
img_height = 10000

In [7]:
model = Sequential([
    layers.Rescaling(1./255, input_shape=(img_width, img_height, 3)),
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(2)
    ])
 
opt = tf.keras.optimizers.SGD(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])
 
earlystop = EarlyStopping(patience=7)   
callbacks = [earlystop]

model.summary()

2022-08-31 15:13:35.610521: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 500, 10000, 3)     0         
                                                                 
 conv2d (Conv2D)             (None, 500, 10000, 16)    448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 250, 5000, 16)    0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 250, 5000, 32)     4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 125, 2500, 32)    0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 125, 2500, 64)     1

In [8]:
GroupSize = 1000
Groups = min(int(np.size(Good_files) / GroupSize),  int(np.size(Bad_files) / GroupSize))

In [9]:
i = 0
loopSize = GroupSize

tg = Parallel(n_jobs=num_cores)(delayed(addImage)(file) for file in Good_files[0:1000]) #i*loopSize:(i+1)*loopSize])
tb = Parallel(n_jobs=num_cores)(delayed(addImage)(file) for file in Bad_files[0:1000]) #i*loopSize:(i+1)*loopSize])

In [10]:
Images = []
Truth = []

for j in tg:
    Truth.append(j[0])
    Images.append(j[1])
for j in tb:
    Truth.append(j[0])
    Images.append(j[1])

Images=np.asarray(Images)
Truth=np.asarray(Truth)

In [11]:
x_train, x_test,y_train,y_test = train_test_split(Images,Truth, test_size=0.15, shuffle=True, random_state=0)

y_train = np.matrix(y_train)
x_train = np.asarray(x_train)

In [12]:
np.max(Truth)


1

In [13]:
history = model.fit(x = x_train, y = y_train, epochs=1, batch_size = 32 , shuffle=True, validation_split=0.25, callbacks=callbacks)

ValueError: in user code:

    File "/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/lib/python3.9/site-packages/keras/engine/training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/lib/python3.9/site-packages/keras/engine/training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/lib/python3.9/site-packages/keras/engine/training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/lib/python3.9/site-packages/keras/engine/training.py", line 860, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/sciclone/home20/dchendrickson01/.conda/envs/tfcgpu/lib/python3.9/site-packages/keras/engine/training.py", line 918, in compute_loss
        return self.compile

In [ ]:
for k in range(Groups):
    
    history, model = loopLearning(k, GroupSize, model)
    print(k)

In [ ]:
model.save(imFolder)

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig(imFolder + 'ModelAccuracy.png')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig(imFolder + 'ModelLoss.png')
plt.show()